In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

import urban_dictionary_scraper
import logging
import pickle
from tqdm.notebook import tqdm
from collections import OrderedDict
from functools import partial
from concurrent.futures import ThreadPoolExecutor
from multiprocessing.pool import ThreadPool

In [4]:
logging.basicConfig(level=logging.INFO)
session = urban_dictionary_scraper.get_session(throttle=0.001, expiry = (7*24*3600))

In [ ]:
all_urls = urban_dictionary_scraper.fetch_all_word_urls(session)

2072755

In [13]:
with open("all_urls.pickle", "wb") as f:
    pickle.dump(all_urls, f, pickle.HIGHEST_PROTOCOL)

In [3]:
with open("all_urls.pickle", "rb") as f:
    to_fetch = pickle.load(f)
    
with open("all_words.pickle", "rb") as f:
    already_done = pickle.load(f)
    for key in already_done.keys():
        del to_fetch[key]

In [10]:
len(already_done)

46784

In [5]:
def fetch_all_definitions(session, to_fetch, already_done=None, save_interval=1000, save_path="all_words.pickle", executor=None):
    already_done = already_done if already_done is not None else OrderedDict()
    fetch_list = list(to_fetch.values())
    pbar = tqdm(total=len(to_fetch))
    
    mapper = executor.imap_unordered if executor else map
    for i, word in enumerate(mapper(partial(urban_dictionary_scraper.fetch_word, session), (e.url for e in fetch_list))):  
        assert word.title in already_done or word.title in to_fetch
        already_done[word.title] = word
        del to_fetch[word.title]
        pbar.update()
    
        if i % save_interval == 0:
            with open(save_path, "wb") as f:
                pickle.dump(already_done, f, pickle.HIGHEST_PROTOCOL)
    pbar.close()

2025971

In [8]:
t = ThreadPool(4)
#with ThreadPoolExecutor(max_workers=5) as executor:
try:
    fetch_all_definitions(session, to_fetch, already_done, executor=t)    
finally:
    t.terminate()
    t.join()

KeyboardInterrupt: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options

In [ ]:
url = "https://www.urbandictionary.com/browse.php?character=A"

# create a new Firefox session
options = Options()
options.set_headless(headless=True)
driver = webdriver.Firefox(firefox_options=options)
driver.implicitly_wait(30)
driver.get(url)

In [ ]:
BeautifulSoup(driver.page_source)